<a href="https://colab.research.google.com/github/Mjohn41823/Deep-Learning-Text-Search/blob/main/Deep%20Learning%20Text%20Output_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LSTM text generation

In [2]:
import keras
import numpy as np
from tensorflow import keras as tf

path = 'Macbeth.txt'
text = open(path).read().lower()
print('Corpus Length:', len(text))

Corpus Length: 116578


Vectorizing Sequences

In [3]:
maxlen = 60

step = 3

sentences = []

next_chars = []

for i in range(0, len(text) - maxlen, step):
  sentences.append(text[i: i + maxlen])
  next_chars.append(text[i + maxlen])

print('Number of Sequences:', len(sentences))

chars = sorted(list(set(text)))
print("Unique Characters", len(chars))
char_indices = dict((char, chars.index(char)) for char in chars)
print('Vectorization')

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
      x[i,t,char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

  

Number of Sequences: 38840
Unique Characters 65
Vectorization


Single layer model for next character prediction

In [4]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))


Model configuration

In [5]:
optimizer = keras.optimizers.adam_v2
model.compile(loss='categorical_crossentropy')

Function to sample the next character give the model's prediction

In [6]:
def sample(preds, temperature=1.0):
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)
  

Text Generation Loop

In [11]:
import random
import sys

for epoch in range(1, 20):
     print('epoch', epoch)
     model.fit(x, y, batch_size=128, epochs=1)
     start_index = random.randint(0, len(text) - maxlen - 1)
     generated_text = text[start_index: start_index + maxlen]
     print("-Generating with seed")

     for temperature in [0.2, 0.5, 1.0, 1.2]:
       print("temperature:", temperature)
       print(generated_text,"\nNext Loop\n", file=open('output.txt', 'a'))

for i in range(400):
  sampled = np.zeros((1, maxlen, len(chars)))
  for t, char in enumerate(generated_text):
    sampled[0, t, char_indices[char]] = 1
  preds = model.predict(sampled, verbose=0) [0]
  next_index = sample(preds, temperature)
  next_char = chars[next_index]
  generated_text += next_char
  generated_text = generated_text[1:]
  print(next_char,"\nNext Loop\n", file=open('output.txt', 'a'))


epoch 1
304/304 [==============================] - 59s 193ms/step - loss: 0.9123
-Generating with seed
temperature: 0.2
temperature: 0.5
temperature: 1.0
temperature: 1.2
epoch 2
304/304 [==============================] - 58s 192ms/step - loss: 0.8927
-Generating with seed
temperature: 0.2
temperature: 0.5
temperature: 1.0
temperature: 1.2
epoch 3
304/304 [==============================] - 58s 192ms/step - loss: 0.8755
-Generating with seed
temperature: 0.2
temperature: 0.5
temperature: 1.0
temperature: 1.2
epoch 4
304/304 [==============================] - 58s 192ms/step - loss: 0.8597
-Generating with seed
temperature: 0.2
temperature: 0.5
temperature: 1.0
temperature: 1.2
epoch 5
304/304 [==============================] - 58s 192ms/step - loss: 0.8431
-Generating with seed
temperature: 0.2
temperature: 0.5
temperature: 1.0
temperature: 1.2
epoch 6
304/304 [==============================] - 58s 192ms/step - loss: 0.8263
-Generating with seed
temperature: 0.2
temperature: 0.5
temperat